<a href="https://colab.research.google.com/github/jinyang628/chineseMultiClassification/blob/main/chineseMultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.5 MB/s eta 0:00:0

In [89]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import re
import string
import nltk
from collections import Counter
from datasets import load_dataset
import jieba

dataset = load_dataset("seamew/THUCNews")
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words("chinese"))

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])

In [123]:
def remove_noise(text):
    chinese_punctuation = ["，","。","！","？","；","：","「","」","『","』","【","】","（","）","《","》","〈","〉","·","“","”","‘","’","(",")"]
    segmentedText = list(jieba.cut(text))
    no_punctuation = [word for word in segmentedText if word not in chinese_punctuation]
    no_filler_words = [word for word in no_punctuation if word not in stop]
    filtered_words = [word for word in no_filler_words if not re.match(r'^\d+(?:[.,]\d+)?[%]?$|^[\d.,]+$', word)]
    return "".join(filtered_words)

df_train["text"] = df_train["text"].map(remove_noise)
df_val["text"] = df_val["text"].map(remove_noise)

In [127]:
def counter_word(dataset):
    counter = Counter()

    for example in dataset:
        tokens = jieba.lcut(example)
        counter.update(tokens)

    return counter

counter = counter_word(df_train["text"])
num_unique_words = len(counter)

In [132]:
train_sentences = [example for example in df_train['text']]
train_labels = [example for example in df_train['label']]
val_sentences = [example for example in df_val['text']]
val_labels = [example for example in df_val['label']]

In [133]:
tokenizer = Tokenizer(num_words=(num_unique_words + 1), oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)


In [135]:
word_index = tokenizer.word_index

In [138]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)


In [139]:
max_length = max(max(len(seq) for seq in train_sequences),
                 max(len(seq) for seq in val_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post')

In [140]:
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [141]:
# If same output, everything is right so far
# print(decode(train_sequences[10]))
# print(train_sentences[10])

深圳小产权房预期领准生证待转房闻风涨价
深圳小产权房预期领准生证待转房闻风涨价


In [155]:
model = keras.models.Sequential()
model.add(Embedding(num_unique_words, 32, input_length=max_length))
# dropout parameter of 0.1 specifies that a fraction of the input units (10%) will be randomly set to 0 during training to prevent overfitting.
model.add(LSTM(64, dropout=0.1))
# 10 because there are 10 different labels 
model.add(Dense(10, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5, 32)             1823936   
                                                                 
 lstm_2 (LSTM)               (None, 64)                24832     
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,849,418
Trainable params: 1,849,418
Non-trainable params: 0
_________________________________________________________________


In [156]:
train_labels_onehot = tf.keras.utils.to_categorical(train_labels)
val_labels_onehot = tf.keras.utils.to_categorical(val_labels)

In [157]:
loss = keras.losses.CategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy"]
model.compile(loss=loss, optimizer=optim, metrics=metrics)
model.fit(train_padded, train_labels_onehot, epochs=2, validation_data=(val_padded, val_labels_onehot), verbose=2)

Epoch 1/2
1563/1563 - 46s - loss: 2.2989 - accuracy: 0.1061 - val_loss: 2.2946 - val_accuracy: 0.1072 - 46s/epoch - 30ms/step
Epoch 2/2
1563/1563 - 40s - loss: 1.6418 - accuracy: 0.3608 - val_loss: 2.4322 - val_accuracy: 0.1030 - 40s/epoch - 25ms/step


In [158]:
predictions = model.predict(val_padded)
"""
For multiclassification the predicted probabilities from the model will be in 
the form of a 2D array, where each row represents a sample and each column 
represents the probability of that sample belonging to a particular class
"""
predicted_labels = np.argmax(predictions, axis=1)

157/157 [==============================] - 1s 2ms/step


In [160]:
print(val_sentences[0:5])
print(val_labels[0:5])
print(predictions)

['++韦德一条龙暴扣三巨头还', '木乃伊今晨公映大片元素成热议话题图', '匈牙利岁女贼落网爱乘飞机四处作案', '男子解救被劫朋友反遭殴打绑架图', '倍光学变焦镜头三星WB550仅售']
[0, 1, 7, 7, 8]
[[0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.04003971]
 [0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.04003971]
 [0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.04003971]
 ...
 [0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.04003971]
 [0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.0400397 ]
 [0.07852808 0.09385665 0.10299557 ... 0.083103   0.20188595 0.0400397 ]]
